In [ ]:
%pip install langchain_openai langchain langchain_core

In [ ]:
import json
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

chat = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_template(
    "Generate {number_of_qs} multiple choice questions for a quiz on the topic of {topic}. Each question should have 4 possible answers, with one correct answer. The questions should be of medium difficulty. The response should json supported. Think step by step and don't send an incomplete response.")

class MCQQuestion(BaseModel):
    """ Response format for a multiple choice question """
    question: str = Field(description='The question text')
    answers: list[str] = Field(description='A list of 4 possible answers')
    correct_answer: str = Field(description='The correct answer')

class MCQQuestions(BaseModel):
    """ A list of multiple choice questions """
    list_of_questions: list[MCQQuestion] = Field(
        description='A list of multiple choice questions')

llm = chat.with_structured_output(MCQQuestions)
chain = prompt | llm

In [ ]:
topics = json.loads(open("topics.json").read())['topics']

all_questions = {}
for topic in topics:
    response = chain.invoke({"number_of_qs": 50, "topic": topic})
    all_questions[topic] = response.list_of_questions

In [ ]:
questions = []
for topic, questions_list in all_questions.items():
    for question in questions_list:
        questions.append({
            'topic': topic,
            'question': question.question,
            'answers': question.answers,
            'correct_answer': question.correct_answer
        })

In [ ]:
df = pd.DataFrame(questions)
df.to_csv("mcq_questions.csv", index=False)